In [24]:
from tqdm import tqdm
import numpy as np

import vae
import data

In [6]:
128*128*128

2097152

In [25]:
# Creates shape random shape data.
shape_data = data.Generator(50000).generate_shapes()

# Creates PyTorch dataset.
dataset = vae.ShapeData(shape_data)

# Build VAE Model.
model = vae.VAE(2)

# Initialises model trainer.
trainer = vae.Trainer(dataset,model,lr=1e-3,batch_size=128,loss='mse')
trainer.train_model(1000)

100%|██████████| 50000/50000 [00:05<00:00, 8550.00it/s]


Epoch 1, Loss: 50170.493980407715
Epoch 2, Loss: 48394.25939941406
Epoch 3, Loss: 47078.96765136719
Epoch 4, Loss: 46151.37939453125
Epoch 5, Loss: 44028.141456604004
Epoch 6, Loss: 44706.367851257324
Epoch 7, Loss: 44652.23751831055
Epoch 8, Loss: 44222.470264434814
Epoch 9, Loss: 44081.43756866455
Epoch 10, Loss: 44605.63494873047
Epoch 11, Loss: 43708.80554962158
Epoch 12, Loss: 43974.443908691406
Epoch 13, Loss: 43977.9345703125
Epoch 14, Loss: 44212.03372955322
Epoch 15, Loss: 44126.36290740967
Epoch 16, Loss: 43662.07482910156
Epoch 17, Loss: 43860.56167984009
Epoch 18, Loss: 42744.20090484619
Epoch 19, Loss: 43043.962841033936
Epoch 20, Loss: 43265.62046813965
Epoch 21, Loss: 42657.52530288696
Epoch 22, Loss: 42988.1629447937
Epoch 23, Loss: 43473.53494262695
Epoch 24, Loss: 42920.80563354492
Epoch 25, Loss: 43489.30244445801
Epoch 26, Loss: 42318.63516998291
Epoch 27, Loss: 42914.56800842285
Epoch 28, Loss: 42997.71014404297
Epoch 29, Loss: 42590.65693664551
Epoch 30, Loss: 421